# VinBigData Chest X-ray Abnormalities Detection

https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection

14の異常所見の位置検出

<img src="https://i.imgur.com/QWmbhXx.png">

### Dataset

- データの作成方法
    - Train
        - 15,000枚
        - 3人の読影医が他の読影医のアノテーションを見ないようつける	
    - Test
        - 3,000枚
        - 5人の読影医の合意のもと決定
            - 最初の3人がまずアノテーションし，よりベテランの読影医2名がレビューし最終決定する
- TrainとTestの比較
    - 年齢が若干Testのほうが低い
- 論文
    - cf. https://arxiv.org/pdf/2012.15029.pdf
    - データセットの作成方法記載
- 外部データ
    - https://cloud.google.com/healthcare/docs/resources/public-datasets/nih-chest
    - https://www.kaggle.com/nih-chest-xrays/data?select=BBox_List_2017.csv
    - https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia
 
### 評価指標

- IoU > 0.4 & クラス分類
    - cf. https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/discussion/212287
    - スコア計算
        - https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocotools/cocoeval.py


## ❏ 予備知識

### 14の異常所見

0. `Aortic Enlargement` - 大動脈肥大所見．高血圧・動脈硬化など血管に負荷がかかっている患者で見られる．
1. `Atelectasis` - 無気肺．気管支の閉塞により閉塞部より末梢の肺が虚脱する．閉塞の原因としては癌や異物の誤嚥など．
2. `Calcification` - 石灰化．病的なものも一部あるが，基本的には加齢による変化みたいなイメージがあります．
3. `Cardiomegaly` - 心陰影拡大．何らかの心疾患を示唆．心陰影の最大横径が胸郭の半分以上であった場合に心拡大陽性と呼ぶ．
4. `Consolidation` - 浸潤影．内部の血管陰影が見えないほど濃い異常陰影．肺炎などがあると出現する．
5. `ILD` - 間質性肺炎(Interstitial Lung Disease)．背景としては膠原病，薬剤，原因不明など多岐にわたる．5の浸潤影と対になって「すりガラス影」と呼ばれる陰影が見られることがある．両者の違いは陰影の濃さで，内部の血管陰影が透見できる程度に薄い陰影をすりガラス影，血管陰影が潰れるくらい濃い陰影を浸潤影と呼ぶ．
6. `Infiltration` - これは正直初耳．consolidationと少しオーバーラップした概念かも？
7. `Lung Opacity` - 肺野異常陰影．これも語彙としては曖昧． 5, 6, 7, 9, 11, 12, 14なども異常陰影の一種なのでこれは分け方として微妙なんじゃないか？
8. `Nodule/Mass` - 結節/腫瘤．3cmより小さいものが結節，3cmより大きいものが腫瘤．癌とか結核とかアスペルギローマというカビ感染とか．
9. `Other Lesion` - その他．皮下気腫とか肋骨骨折とかがここらしい．
10. `Pleural Effusion` - 胸水．胸腔内に水が貯まった状態．肺炎や癌などがひどいと生じる．
11. `Pleural Thickening` - 胸膜肥厚．アスベスト肺やそれに伴う中皮腫(胸膜の癌)などで観察されそうなイメージ．
12. `Pneumothorax` - 気胸．自然気胸，外傷性気胸などが存在．若くて痩せた高身長の男性に多く通称"イケメン病"
13. `Pulmonary Fibrosis` - 肺線維症．間質性肺炎と重複部分をもつ概念．肺の線維化が進行すると命が危ないイメージ

- いくつかのクラスが重複していそうということが別のディスカッションでも書かれている
    - cf. https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/discussion/207750

### 物体検出

- model
    - Yolo v5
    - Efficient Det
    - Sparse R-CNN
        - paper: https://arxiv.org/pdf/2011.12450.pdf
    - MMDetection
        - https://github.com/open-mmlab/mmdetection
    - Detectron2
        - cf. https://github.com/facebookresearch/detectron2
- boxes fusion(VinBigData - Fusing Bboxes + Coco Dataset)
    - Non-maximum Suppression(NMS)
    - Soft-NMS
    - Non-maximum Weighted(NMW)
    - Weighted Bboxes Fusion(WBF)
- EMA(exponential moving average)
    - https://arxiv.org/pdf/1904.06031.pdf


## ❏ 当該コンペ特有の知識

- [Chest_X-ray: 14の異常所見の基礎知識](https://www.kaggle.com/sakuraandblackcat/chest-x-ray-14)

## EDA

- 異常なしを含む各異常所見数



<img src="https://www.kaggleusercontent.com/kf/50753336/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..a5GBM-CGjzPK4BKGkU8oaQ.TjFf1eUGXd9g-Mm15xbmDCc4JT-7Ac17PiTVaw8isaVrlL77Q3rl6bhcJJag7-ytRQPsi6nsDgftsbL2HcBNlXckRIKeP5Qkq2xseqggJV0V2_3zr9K6HV5S841CdVI8xMTSxBBsm9L9FJFkVTUm5MA1SZkxcndQ2oXuZjEBoTfdEay830PieceA41yeEuq-sx9ODI-lTbfNi0KnSFW5uV2DL-Zf-yyZksenSxu5-VJYa9Wsz3PnfJNnD9lCJWXmSmeuQGTHXltBLy13kKjhvv7lDMvRj0r4KBicLex4qamb-_D3K5vwuYkSDONVMLCUwXGxm_YrPrGTjycH5AGQy3BlXvUy8N7kqMOZsOKGuk0Mb3q3bAZH0co8N0QIOS27SSV6LNT7lwRx9rGieJ6ATT0G0s5mBjbPlo2A3Vj33ahOGnYm1dLady8aUhUjzycBkQ1vx4IXS8mZ-J2qr3LgGHclRdrKun3A9PRF0DyOsxiDzrRxXOKLVIZyD4iFdiPSQbXffhPeMh8tJmrQtK8s3Go7GOUM61XRKdGlqfSlgu-6iITO8c8Hfd11W690eeNzZXT5EzjekOOu8gtf4esviKcZIElk3g9H-nlfxr6Cj9zwYnSyVDJfPNMbsxexfH1CU0QgsBME0VysWQvOmbes98sxt0Wn0Ijsa1TRvBsVxXqT_Mcph2VnkaIfd8sq5aeqH4XJBK2Ux0C3tW40aGCLtw.kbIqxfJV7yt90HCyYPP6Lg/__results___files/__results___16_0.png" width="600" />

<img src="https://www.kaggleusercontent.com/kf/53546324/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..GffM2aPaOjb_u4HvFGAHXA.I_BgSleAanujogZ-UmWuaqa_QO8FD5QOo3xbJB2rDc9AJvZdrAlM_aFKGlaaZeMJGMyeK_vm7zUWaJaiQoNIRKPfFHNJRPrRsgD-7GnCN2GSzmsy5eRHM_T0o1YGbQxuWXneG0Yu4ibsDbKu8olxc3HNKWlWksFrYhakmf2RDIq3LAAquWxlsFzyhOH6a6AppW9sqcFWs48fJyfVcXAiJUpDk2DlGNmMUSEmXn87f-_c5Qs-gjBdYpLD9uIP4j_e4s_uGM8Owr2DxsqTg9b49bKZ4SZpbZx3Et-c_2BXeXp6st_A01714Xgmrl8OZlvADU_9kqUTBuWQ28NlihXCnlxwdXN6ejugrSvGWVmal3OkvSXp8AZ0HBiKn_jCbw6ta4c-tU_4Hqr_ZegC4w61PDNZRNES4rKI-gdX5q8ImtL6THhp86yYMrOBjgMQjkjJNZ5t3IE_kuXjn_iCXyCyQs_taNHg_rH1Ws9yRpcon1KzTsRWCzzWaAMRPkNrjXTJUDcm9eCmHnl7DB4j6Yvhh4E0GTK8lMugi1nYk6DUzWeCbpFZSw9Da1eF0zxJAwvH2eFrXWukkuyg-iYqj9SO9cPOYq5t_Dqotwky7cnMAUkTz1p-xy3mj01_tlwUZpaDe2hYK8P4d18rwpmdyERdr-NfsH6bEGrBbYya2iGnMr3UgBPu-vcEhz9hriXk1tQeWvC1JotI7eVnmdSPEy1r-A.6RyLVwAcriXv8tnnoAyQPg/__results___files/__results___76_0.png" width="600" />

- cf. https://www.kaggle.com/dschettler8845/visual-in-depth-eda-vinbigdata-competition-data
    - 結節全体を囲っている方もいるので，どっちに合わせるか考えたほうが良いかも(R10単体の意見で性能がよかったらしいので，それを参考にする?)

## 全体的な方針

- 物体検出+２クラス分類問題として処理する
    - 2クラス分類
        - 異常所見を含む画像かどうか
        - cf. https://www.kaggle.com/corochann/vinbigdata-2-class-classifier-complete-pipeline
    - 物体検出
        - どのような異常所見がどの部分に含まれるか
        - cf. https://www.kaggle.com/corochann/vinbigdata-detectron2-train
        - cf. https://www.kaggle.com/corochann/vinbigdata-detectron2-prediction
- pipeline
    - 異常所見を含む胸部X線画像かどうかを判定する
    - 異常所見のあると推定された画像に対して物体検出を行う
- model
    - Yolo v5
    - EfficientDet
- starter kit
    - https://www.kaggle.com/nvnnghia/yolov5-pseudo-labeling/data
        - 小麦の穂の位置検出コンペが有用そう

## スターターキット


- training
    - https://www.kaggle.com/awsaf49/vinbigdata-cxr-ad-yolov5-14-class-train/data
        - Yolo v5
    - https://www.kaggle.com/corochann/vinbigdata-detectron2-train
        - Detectron2
        
- https://www.kaggle.com/nvnnghia/yolov5-pseudo-labeling/data
    - コンペは違うけど，Yolo v5での実装例あり
    - pseudo labelingが小麦コンペでは有効だった


### 関連コンペ(直近の物体検出コンペ)

- クラス分類
    - [Cassava Leaf Disease Classification](https://www.kaggle.com/c/cassava-leaf-disease-classification/overview/evaluation)
        - キャッサバの葉の病変分類
- 物体検出
    - [Global Wheat Detection](https://www.kaggle.com/c/global-wheat-detection/overview)
        - 小麦の穂の位置検出
    - [NFL 1st and Future - Impact Detection](https://www.kaggle.com/c/nfl-impact-detection/overview)
        - ヘルメットへの衝撃検出コンペ(動画からの物体検出)
    - [RSNA肺炎検出チャレンジ](https：//www.kaggle.com/c/rsna-pneumonia-detection-challenge/)
    - [SIIM-ACR気胸セグメンテーション](https：//www.kaggle.com/c/siim-acr-pneumothorax-segmentation)
- Kaggle以外
    - CheXpert：https：//stanfordmlgroup.github.io/competitions/chexpert/
    - CheXphoto：https：//stanfordmlgroup.github.io/competitions/chexphoto/

### 参考になりそうなコード

- [score:0.235](https://www.kaggle.com/nxhong93/yolov5-chest-512)
    - 推論部分のみ(学習部分はない)
    - dataset
        - image resolution
            - 512x512
    - model
        - yolo v5
- [score:0.239](https://www.kaggle.com/prashantkikani/vinbigdata-post-processing)
    - 異常なしを含む提出ファイルのアンサンブル方法
- [score:0.260](https://www.kaggle.com/klawensliu/mmdetection-0-260-baseline)
    - 推論部分のみ?
    - 学習にA100 x 2を利用(原寸サイズでの学習にはGPUのメモリが30GB以上必要とのこと)
    - MMDetectionの使い方記載
- [score:0.161 - 0.175](https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/discussion/213462)
    - dataset
        - image resolution
            - 1024x1024
    - model
        - VFNet(mmdet)
    - train
        - augmentation
            - rotation, shift, brightness/contrast, equalize
        - boxes fusion
            - nms
    

### 参考になりそうなディスカッション

#### 前処理

- https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/discussion/207955
    - リサイズされた画像置き場
- [opular X-ray image normalization techniques](https://www.kaggle.com/raddar/popular-x-ray-image-normalization-techniques)
    - 胸部X線画像の正規化

#### 後処理

- https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/discussion/211971
    - <font color="red">正常なのかどうか微妙なラインの場合は"14 1 0 0 1 1"を追加することでスコア改善</font>
        - さらに，すべての画像に対して，正常である確率を追加してあげると性能が上がる模様
            - cf. https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/discussion/219672
    - 正常の場合は画像左上に1ドットの矩形扱いにしているため生じるIoUスコア計算上の問題でバグではない

### 役立ちそうなライブラリ

- https://github.com/Klawens/dataset_prepare
    - 外部データを含む各種データセットをCOCO形式に変換するライブラリ
- https://www.kaggle.com/angqx95/weightedboxfusion
    - WBF(Weighted boxes fusion)
    - 物体の位置検出情報のアンサンブルライブラリ
- [MMDetection](https://github.com/open-mmlab/mmdetection)
    - 様々な物体検出アルゴリズムを揃えたツールボックス
        - ただし，Yolo v5やEfficientDetなど新しいアルゴリズムは今の所ない
- [Sparse R-CNN](https://github.com/PeizeSun/SparseR-CNN)
     - Sparse R-CNN

## ❏ References

- [【日本語】Chest_X-ray: 14の異常所見の基礎知識](https://www.kaggle.com/sakuraandblackcat/chest-x-ray-14)
- [前処理されたデータセット置き場](https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/discussion/207955)
	- 256, 512, 1024にリサイズ済みのPNG画像を共有
- [VinDr-CXR: An open dataset of chest X-rays with radiologist’s annotations](https://arxiv.org/pdf/2012.15029.pdf), 2021.
   - データセットに関する論文
- https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/discussion/208837
    - LB: 0.155達成時の解法
        - detection: sparse rcnn
        - classification: resnet-50
- [Previous X-Ray Competitions on Kaggle](https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/discussion/207664)
    - 胸部X線の過去コンペ情報